In [4]:
from load_mock_data import load_mock_data
load_mock_data()

In [6]:
from pathlib import Path

import gspread
import pandas as pd
import snowflake.connector
from oauth2client.service_account import ServiceAccountCredentials
from snowflake.connector.pandas_tools import write_pandas
from utils import load_config

PATH_GSHEET_CREDS = Path("/Users/selimrbd/.gsheet/mock-data-412317-cd4e142a8ff3.json")

# Set up the credentials
scope = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
creds = ServiceAccountCredentials.from_json_keyfile_name(PATH_GSHEET_CREDS, scope)
client = gspread.authorize(creds)

# Google Sheets config
sheet_url = "https://docs.google.com/spreadsheets/d/1OEe9yHiqlQXbCzP6JA7HkBOdBOpuw6RtNHmyqRF4lUI/edit#gid=808477594"
spreadsheet_id = sheet_url.split("/d/")[1].split("/")[0]
spreadsheet = client.open_by_key(spreadsheet_id)


cfg = load_config()["snowflake"]

# Snowflake config
DATABASE = "RAW"
SCHEMA = "MOCK"


def load_mock_data():
    snow_ctx = snowflake.connector.connect(
        user=cfg["username"],
        password=cfg["password"],
        account=cfg["account"],
        warehouse=cfg["warehouse"],
        database=DATABASE,
        schema=SCHEMA,
    )

    for sheet in spreadsheet.worksheets():
        # Read data into a pandas DataFrame
        data = sheet.get_all_values()
        df = pd.DataFrame(data)
        df.columns = df.iloc[0]  # Set the first row as column headers
        df = df.drop(0).reset_index(drop=True)

        # Convert column names to uppercase to align with Snowflake's default behavior
        df.columns = [column.upper() for column in df.columns]
        return df
        # Define the Snowflake table name
        table_name = f"{sheet.title}"

        # Load DataFrame into Snowflake
        write_pandas(
            snow_ctx, df, table_name.upper(), auto_create_table=True, overwrite=True
        )

    # Close Snowflake connection
    snow_ctx.close()


if __name__ == "__main__":
    load_mock_data()


In [7]:
dd = load_mock_data()

In [10]:
dd

,URL,MEASURED_AT_TS,P50_RESPONSE_TIME,NB_REQUESTS
0,/api/courses,2024-01-23,2000,23450
1,/api/courses,2024-01-24,45,12983
2,/api/courses,2024-01-25,21,98112
3,/api/projects,2024-01-23,89,11872
4,/api/projects,2024-01-24,77,98734
5,/api/projects,2024-01-25,34,11934
6,/courses/id,2024-01-23,12,71273
7,/courses/id,2024-01-24,23,88172
8,/courses/id,2024-01-25,45,11623
